## **Pre-read for RAG Module Sessions 1 & 2: Building the Retrieval Foundation**

This document provides a high-level overview of the first two sessions of the Retrieval-Augmented Generation (RAG) module, covering necessary installations, topics, and key learning objectives. We will focus on building the essential data preparation and retrieval components of a RAG system, with hands-on work centered around using the **Qdrant vector database** in its **local, in-memory configuration** via the `qdrant-client` library.

### **Installation Prerequisites**

To follow along with the practical exercises, you will need Python (3.8+ recommended) and the following libraries installed. We highly recommend using a virtual environment.

Run the following command in your terminal:

```shell
    pip install langchain-text-splitters langchain-community langchain-huggingface langchain-qdrant sentence-transformers qdrant-client numpy scikit-learn rank_bm25
```

This command installs libraries for:

* Text splitting (`langchain-text-splitters`).  
* LangChain integrations for embeddings and vector stores (`langchain-community`, `langchain-huggingface`, `langchain-qdrant`).  
* Generating vector embeddings (`sentence-transformers`).  
* Interacting with Qdrant locally (`qdrant-client`).  
* Numerical operations (`numpy`).  
* Keyword search concepts (`scikit-learn` for TF-IDF, `rank_bm25` for BM25).

**Note on Qdrant:** For these sessions, we will use the `qdrant-client` library to run Qdrant entirely in your local process memory (`location=":memory:"`). This requires **no separate Qdrant server installation or Docker setup**.

### **Session 1: Foundations \- Introduction to RAG and Document Preparation**

**Session Goal:** Introduce the core concepts of RAG, explain why it's necessary, outline its workflow, and delve into preparing your data by chunking documents. We will also introduce the basics of keyword search and the fundamental idea of vectorization for semantic understanding.

**Topics You Will Learn:**

* **What is RAG and Why Use It?** Understand RAG as an architectural pattern to enhance LLMs by giving them access to external knowledge. Learn how RAG helps overcome LLM limitations like static knowledge, hallucination, and lack of access to private data. Explore various use cases.  
* **Core RAG Architecture & Workflow:** Identify the main components: External Knowledge Source (Vector Database), Retriever, and Generator (LLM). Understand the Ingestion (data preparation, indexing) and Inference (query processing, retrieval, generation) stages.  
* **Document Preparation: Chunking:** Learn why large documents must be split into smaller "chunks" due to LLM context window limitations. Understand that chunking strategy impacts retrieval quality.  
* **Chunking Strategies:** Cover practical methods using LangChain:  
  * **Fixed-Size Chunking:** Splitting by a fixed character count, with optional overlap.  
  * **Recursive Character Text Splitting:** Splitting using a list of separators to preserve semantic units.  
  * *(Other strategies like Semantic or Document-Based chunking will be mentioned conceptually)*.  
* **Keyword Search Fundamentals:** Basic concepts like TF-IDF and BM25 will be introduced as traditional methods relying on exact word matching. Contrast this with semantic understanding.  
* **Introduction to Vectorization:** Learn how text is converted into numerical vectors (embeddings) that capture meaning. Understand that texts with similar meanings are close in vector space. Introduction to **Transformer models** and **Sentence Transformers** like `all-MiniLM-L6-v2` for creating these embeddings.

**Key Takeaways:**

After Session 1, you should be able to:

* Explain the purpose and workflow of a RAG system.  
* Understand the need for and challenges of document chunking.  
* **Implement fixed-size and recursive text splitting using LangChain**.  
* Explain the difference between keyword search and semantic search.  
* Understand the concept of vector embeddings and how they represent text meaning numerically.

In [2]:
# Installing Prereqs
!pip install -qq langchain-text-splitters langchain-community langchain-huggingface langchain-qdrant sentence-transformers qdrant-client numpy scikit-learn rank_bm25


The pre-req steps to use the Gemini API key with Python:

1.  **Get API Key:** Go to [aistudio.google.com](https://aistudio.google.com/), sign in, and create/copy your API key (secure it immediately).
2.  **Install Library:** In your terminal, run `pip install -q -U google-generativeai`.
3.  **Set API Key Securely:**
    * **Recommended:** Set it as an environment variable (e.g., `export GOOGLE_API_KEY="YOUR_KEY"` in terminal/shell config).
    * **Colab:** Use Colab Secrets to store `GOOGLE_API_KEY`.
4.  **Configure in Python:** In your script, retrieve the key (e.g., `os.getenv('GOOGLE_API_KEY')` or `userdata.get('GOOGLE_API_KEY')`) and then use `genai.configure(api_key=YOUR_KEY)`.

[Gemini API Key](https://aistudio.google.com/u/1/apikey)

In [1]:
import google.generativeai as genai
from dotenv import load_dotenv  # For loading API key from a .env file
import os
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

model_name = 'gemini-2.0-flash'

model = genai.GenerativeModel(model_name)

/workspace/GenAIEngineering-Cohort1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **What is RAG?**

RAG is an AI framework that enhances the capabilities of LLMs by enabling them to access, retrieve, and incorporate external, up-to-date, and authoritative information into their responses. It combines the strengths of two distinct paradigms:

1. **Retrieval:** The ability to efficiently search and retrieve relevant information from a vast corpus of documents.  
2. **Generation:** The LLM's ability to generate coherent and contextually relevant text.

In essence, RAG acts as a bridge, allowing LLMs to look up information in a knowledge base *before* formulating a response, much like a human would consult a textbook or database to answer a complex question.

### **Why is RAG Needed?**

RAG addresses several critical limitations of standalone LLMs:

1. **Mitigating Hallucinations:** One of the most significant challenges with LLMs is their tendency to invent information, especially when they don't have a confident answer. By grounding their responses in retrieved facts, RAG significantly reduces the likelihood of hallucinations, leading to more reliable and trustworthy outputs.  
2. **Access to Up-to-Date Information:** LLMs have a knowledge cutoff date, meaning they cannot access information beyond their last training update. RAG allows them to query continually updated databases, ensuring their responses are current and accurate, vital for dynamic fields like news, finance, or scientific research.  
3. **Domain-Specific Knowledge:** Pre-trained LLMs are generalists. They lack specific knowledge about a particular company's internal documents, proprietary data, or niche industry information. RAG enables LLMs to tap into these private, domain-specific knowledge bases, making them useful for enterprise-level applications.  
4. **Transparency and Explainability:** When using RAG, the generated answer can often be directly linked back to the source documents from which the information was retrieved. This provides a degree of explainability and allows users to verify the claims, building trust in the AI system.  
5. **Reduced Training Costs:** Instead of constantly re-training or fine-tuning large models on new data (which is incredibly expensive and time-consuming), RAG offers a more agile way to update an LLM's knowledge by simply updating the external knowledge base.

### **How RAG Works (The Process)**

The RAG process typically involves three main steps:

1. **Retrieval:** When a user poses a query (Q), the system first searches a vast external knowledge base (D) (e.g., a vector database containing embeddings of documents, or a conventional search index). The goal is to retrieve the most relevant passages, documents, or snippets (R) related to the query. This step often uses techniques like semantic search, keyword search (BM25), or hybrid approaches.

    R=Retrieve(Q,D)  
2. **Augmentation:** The retrieved information (R) is then concatenated or integrated with the original user query (Q) to form an augmented prompt (). This new, enriched prompt provides the LLM with the necessary context and factual grounding.

    Paug​=Augment(Q,R)  
3. **Generation:** Finally, the augmented prompt (Paug​) is fed into the LLM. The LLM then generates a coherent and accurate answer (A) based on its inherent language understanding capabilities and the specific information provided in the augmented prompt.

    A=Generate(Paug​,LLM)

In essence, RAG transforms an LLM from a black box of pre-trained knowledge into a dynamic, fact-checking, and context-aware reasoning engine, making it a powerful tool for building more reliable and sophisticated AI applications.


In [ ]:
# Without any context

prompt = '''
Given the user query, answer the user to the best of your ability.
Enusre you are polite and helpful.
If you do not know the answer, say "I don't know" and offer to help with something else.

User Query: {user_query}
'''

user_query = 'I need to know which day of the week is the lowest employee attendance.'

response = model.generate_content(prompt.format(user_query=user_query))

print(response.text)

Okay, I understand you'd like to know which day of the week typically has the lowest employee attendance.

Unfortunately, I don't have access to specific data about employee attendance across different companies or industries. The day with the lowest attendance can vary greatly depending on factors like:

*   **Industry:** Some industries might have more weekend work than others.
*   **Company Culture:** Companies with flexible work arrangements might see different patterns.
*   **Location:** Local holidays and events can impact attendance.
*   **Shift Schedules:** Businesses operating 24/7 might have varying attendance based on shift preferences.

**How to Find This Information for Your Specific Situation:**

*   **Check Your Company's Data:** The best source of information would be your company's HR or attendance records. They should be able to provide you with data on employee attendance trends.
*   **Conduct a Survey:** If you don't have access to historical data, you could conduct

In [ ]:
# With Context

import numpy as np
import pandas as pd

attendance = np.random.randint(0, 2, size=(10, 5))

attendance = pd.DataFrame(
    attendance, columns=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
attendance.index.name = 'Employee ID'

context = attendance.to_markdown()


prompt = '''
Given the user query, answer the user to the best of your ability.
Enusre you are polite and helpful.
If you do not know the answer, say "I don't know" and offer to help with something else.

User Query: {user_query}

Context:

{context}
'''

response = model.generate_content(
    prompt.format(
        user_query=user_query,
        context=context,
    )
)

print(response.text)

Okay, I can help you determine the day of the week with the lowest employee attendance based on the data you provided.

Here's the breakdown:

*   **Monday:** 0 + 0 + 1 + 0 + 1 + 0 + 0 + 0 + 0 + 1 = 4
*   **Tuesday:** 1 + 0 + 0 + 0 + 0 + 0 + 0 + 1 + 0 + 1 = 3
*   **Wednesday:** 0 + 1 + 0 + 1 + 0 + 0 + 1 + 0 + 0 + 1 = 4
*   **Thursday:** 1 + 0 + 1 + 1 + 1 + 0 + 0 + 0 + 1 + 1 = 6
*   **Friday:** 1 + 0 + 0 + 1 + 0 + 1 + 0 + 1 + 0 + 1 = 5

Based on these totals, **Tuesday** has the lowest employee attendance with a total of 3.


In [4]:
# Loading Sample Dataset
# https://raw.githubusercontent.com/paraschopra/generating-text-small-corpus/refs/heads/master/all_ml_ideas.csv

corpus = pd.read_csv(
    'all_ml_ideas.csv',
    sep='------',
    engine='python',
    header=None,
    names=['text']
)
corpus

,text
0,Predicting Freeway Congestion
1,Supervised Learning of Query Term Relevant Pro...
2,Robotic Arm Control
3,Network Intrusion Detection
4,Multi-Website Name Coreference Resolution
...,...
2587,Sentimental Analysis with Amazon Review Data
2588,Bird Classification and Feature Recognition
2589,Human or Robot
2590,Predicting NBA shots


## Basic Keyword Search

Keyword search represents the most fundamental and intuitive form of information retrieval, forming the backbone of search engines for decades. At its core, keyword search operates on a simple premise: it matches the specific words (keywords) entered by a user in a query against the words present in a collection of documents. It's often the first interaction point for users seeking information, relying on direct lexical matches rather than deeper semantic understanding.

### **The Core Principle**

When a user inputs a query, a keyword search engine takes these terms and looks for their occurrences within its indexed documents. The primary goal is to identify documents that contain one or more of the specified keywords. For instance, if a user searches for "best Italian restaurant," the system would attempt to find documents containing "best," "Italian," and "restaurant."

To facilitate rapid searching, keyword search engines typically rely on an **inverted index**. An inverted index is essentially a data structure that maps terms to the documents in which they appear. It looks something like this:

* "restaurant" \[Document 1, Document 5, Document 12\]  
* "Italian" → \[Document 1, Document 8, Document 12\]  
* "best" → \[Document 5, Document 12, Document 20\]

When a query comes in, the engine quickly consults this index to retrieve a list of documents containing the query terms.

### **Basic Ranking and Relevance**

Once a set of matching documents is identified, they need to be ranked by relevance. In its simplest form, a keyword search might rank documents based on:

1. Number of Matching Keywords: Documents containing more of the query terms are considered more relevant. For example, a document containing all three terms ("best," "Italian," "restaurant") would be ranked higher than one containing only "Italian" and "restaurant." A very simple conceptual score could be represented as:  
   Let Q=q1​,q2​,…,qm​ be the set of keywords in the query, and D be a document.  
2. Score(D,Q)=i=1∑m​I(qi​∈D)  
3. where I(qi​∈D) is an indicator function that equals 1 if term qi​ is present in document D, and 0 otherwise.  
4. **Keyword Proximity:** Documents where the keywords appear closer together are often ranked higher, as this might indicate a more coherent discussion of the topic.  
5. **Term Position:** Keywords appearing in important fields like the document title, headings, or metadata might be given more weight than those in the body text.  
6. **Boolean Logic:** Users can often refine queries using Boolean operators like AND (all terms must be present), OR (at least one term must be present), and NOT (a term must not be present).

### **Advantages of Keyword Search**

* **Simplicity:** It's easy for users to understand and for developers to implement a basic version.  
* **Speed:** With well-optimized inverted indexes, keyword searches can be incredibly fast, even across vast document collections.  
* **Effectiveness for Precise Queries:** When a user knows the exact terms they are looking for, keyword search can be highly effective in retrieving relevant results.

### **Limitations of Keyword Search**

Despite its widespread use, keyword search suffers from significant limitations:

* **Lack of Semantic Understanding:** This is its most prominent drawback. Keyword search cannot grasp the underlying meaning or intent behind words. It treats "car" and "automobile" as distinct terms, failing to recognize them as synonyms. It also struggles with polysemy (words with multiple meanings, like "bank").  
* **Ignores Word Order and Context:** Unless explicitly using an exact phrase search (e.g., "machine learning" in quotes), keyword search views text as a "bag of words." It cannot distinguish between "man bites dog" and "dog bites man."  
* **Recall and Precision Issues:** It can suffer from low **recall** (failing to retrieve all relevant documents if they use different vocabulary) and low **precision** (retrieving irrelevant documents if keywords are too broad or common).  
* **Spelling Sensitivity:** It is highly sensitive to typos and misspellings; a single incorrect letter can lead to zero results.

In conclusion, keyword search remains a fundamental component of most search systems due to its speed and simplicity. However, its inherent limitations regarding semantic understanding and contextual awareness have driven the development of more sophisticated techniques like TF-IDF, BM25, and ultimately, neural network-based semantic search models that aim to bridge the gap between keywords and true meaning.



In [14]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')


def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    tokens = [
        word for word in tokens if word not in stopwords.words('english')]
    # Remove special characters
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in tokens]
    # Remove empty strings
    tokens = [word for word in tokens if word != '']
    return tokens


search_query = 'What is the best way to learn machine learning?'

preprocessed_query = preprocess_text(search_query)

search_query, preprocessed_query

('What is the best way to learn machine learning?',
 ['best', 'way', 'learn', 'machine', 'learning'])

## The Bag-of-Words (BoW) model

The Bag-of-Words (BoW) model is a foundational and historically significant technique in natural language processing (NLP) and information retrieval. At its core, BoW simplifies text representation by treating a document or query as an unordered collection of its words, essentially like words thrown into a "bag." The key characteristic is that it disregards grammar, syntax, and even the order of words, focusing solely on the presence and frequency of individual words within the text.

### **How Bag-of-Words Works for Search**

For a search engine to utilize a BoW model, it first processes a large collection of documents to build a comprehensive vocabulary – a unique list of all words found across the entire corpus. Each word in this vocabulary becomes a feature or dimension in a vector space.

When a document is processed, it's converted into a vector where each component corresponds to a word in the vocabulary, and its value typically represents the frequency of that word within the document. For example, if the vocabulary contains "cat," "dog," "chase," and "tree," and a document is "The cat chases the dog," its BoW representation might be {"cat": 1, "dog": 1, "chase": 1} (ignoring common words like "the").

When a user submits a search query, that query is also transformed into a similar BoW vector. The search engine then calculates the similarity between the query's vector and the vectors of all documents in its index. Common similarity measures include:

* **Dot Product:** A simple count of shared words.  
* **Cosine Similarity:** Measures the cosine of the angle between two vectors, indicating how similar their directions are. This is often preferred as it's normalized by document length, preventing longer documents from being unfairly favored just because they contain more words.

Documents are then ranked based on their similarity scores to the query, with higher scores indicating greater relevance.

### **Enhancements to the Basic BoW Model**

While the core BoW is straightforward, several enhancements improve its effectiveness:

* **Stop Words Removal:** Common words like "a," "the," "is," "and" are often removed before creating the BoW. These words are frequent but carry little semantic meaning for distinguishing documents. Removing them reduces noise and improves efficiency.  
* **Stemming and Lemmatization:** To account for variations of words (e.g., "run," "running," "ran"), stemming (reducing words to their root, like "run") or lemmatization (reducing words to their dictionary form, like "am," "is," "are" to "be") is applied. This ensures that different grammatical forms of the same word are treated as identical, improving matching accuracy.  
* **TF-IDF (Term Frequency-Inverse Document Frequency):** This is a widely used weighting scheme that goes beyond simple word counts.  
  * **Term Frequency (TF):** Measures how frequently a word appears in a document.  
  * Inverse Document Frequency (IDF): Measures how rare a word is across the entire document collection. Words that are rare but appear in a specific document are considered more important.  
    By multiplying TF and IDF, the BoW model gives more weight to words that are both frequent in a specific document and rare across the entire corpus, thus improving the relevance ranking.

### **Advantages and Limitations**

The Bag-of-Words model offers several advantages:

* **Simplicity and Interpretability:** It's conceptually easy to understand and implement.  
* **Computational Efficiency:** Building and comparing BoW representations is computationally less demanding compared to more advanced methods.  
* **Effectiveness:** Despite its simplicity, it performs reasonably well for many fundamental text classification and retrieval tasks, serving as a strong baseline.

However, BoW also has significant limitations:

* **Loss of Semantic Meaning and Word Order:** This is its most critical drawback. BoW completely ignores the context, grammar, and order of words. For instance, "The dog bit the man" and "The man bit the dog" would have identical BoW representations, even though their meanings are vastly different. It cannot capture phrases or idiomatic expressions.  
* **Synonymy and Polysemy:** It struggles with synonyms (e.g., "car" and "automobile" are treated as distinct words) and polysemy (words with multiple meanings, like "bank" – a river bank vs. a financial institution – are treated as the same word).  
* **High Dimensionality and Sparsity:** For large text corpora, the vocabulary can become enormous, leading to high-dimensional vectors that are mostly empty (sparse), making computations less efficient and potentially impacting accuracy.

In conclusion, while the Bag-of-Words model remains a foundational concept and is surprisingly effective for its simplicity, its inherent limitation of ignoring word order and deeper semantic relationships has paved the way for more sophisticated NLP techniques like word embeddings (Word2Vec, GloVe) and, more recently, transformer-based models (like BERT, GPT, and Gemini), which are designed to capture contextual and semantic nuances of language.


In [6]:
# Brute Force Search with Score based ranking

import re
# regex replace only to keep alphanumeric characters
# and remove special characters
pattern = r'[^a-zA-Z0-9\s]'

search_tokens = re.sub(pattern, '', search_query).lower().split()

print(search_tokens)


def score(text, search_tokens):
    score = 0
    for token in search_tokens:
        if token in text:
            score += 1
    return score


def brute_search(corpus, search_tokens):
    corpus = corpus.copy()
    corpus['score'] = corpus['text'].apply(lambda x: score(x, search_tokens))
    return corpus.sort_values(by='score', ascending=False)


print(brute_search(corpus, search_tokens).head(10).to_markdown())

['what', 'is', 'the', 'best', 'way', 'to', 'learn', 'machine', 'learning']
|      | text                                                                                             |   score |
|-----:|:-------------------------------------------------------------------------------------------------|--------:|
|  807 | An application of machine learning to the board game pentago                                     |       5 |
| 2479 | A machine learning based stock trading framework using technical and economic analysis           |       5 |
| 2124 | Applying machine learning to the board game Pylos                                                |       5 |
|  693 | Early defect identification of semiconductor processes using machine learning                    |       4 |
| 2252 | Uncertainty quantification and sensitivity analysis of reservoir forecasts with machine learning |       4 |
| 1851 | Advanced machine learning techniques for thyroid cancer diagnosis                         

## TF-IDF Vectorization and Search

In the realm of information retrieval and natural language processing, simply counting words (as in the basic Bag-of-Words model) often falls short in determining the true relevance of a document to a search query. This is where TF-IDF, short for Term Frequency-Inverse Document Frequency, emerges as a powerful and widely used statistical measure. TF-IDF provides a numerical statistic that reflects how important a word is to a document in a collection or corpus.

### **Breaking Down TF-IDF**

The TF-IDF score for a word in a document is a product of two components:

1. **Term Frequency (TF):** This measures how frequently a term (word) appears within a specific document. The idea is straightforward: if a word appears many times in a document, it's likely to be important to that document's content. However, simply using raw counts can be misleading, as longer documents will naturally have higher term frequencies for most words. To mitigate this, TF is often normalized, for example, by dividing the raw count by the total number of terms in the document. TF(t,d)=Total number of terms in document dNumber of times term t appears in document d​  
2. **Inverse Document Frequency (IDF):** While TF highlights words frequent within a document, it doesn't account for the overall importance of the word across the entire collection of documents. Words like "the," "a," and "is" appear frequently in almost all documents but carry little unique meaning or discriminative power. IDF addresses this by assigning a higher weight to terms that are rare across the entire corpus and a lower weight to common terms. IDF(t,D)=log(Number of documents d containing term tTotal number of documents N​) The logarithm helps to scale the values. A higher IDF indicates that the term is rare and therefore more significant.

### **The TF-IDF Score and Vectorization**

The TF-IDF score for a term in a document is then calculated by multiplying its TF and IDF values: TFIDF(t,d,D)=TF(t,d)×IDF(t,D)

This combined score assigns a high value to terms that appear frequently in a particular document but rarely in the overall corpus. Conversely, terms that are common across many documents (like stop words) or very rare in a single document will have low TF-IDF scores.

**TF-IDF Vectorization** involves representing each document in a collection as a vector in a high-dimensional space. Each dimension of this vector corresponds to a unique term in the vocabulary of the entire corpus. The value for each dimension in a document's vector is the TF-IDF score of the corresponding term within that document. This process transforms unstructured text into a structured numerical format that machine learning algorithms can easily process.

### **TF-IDF for Search**

When a user submits a search query, the same TF-IDF vectorization process is applied to the query itself. The query is treated as a short document, and its terms are assigned TF-IDF weights based on their frequency in the query and their inverse document frequency across the entire document collection.

Once both the query and all documents in the collection are represented as TF-IDF vectors, search becomes a problem of **vector space model similarity**. The search engine calculates the similarity between the query vector and each document vector. The most common similarity metric used for this is **Cosine Similarity**, which measures the cosine of the angle between two vectors. A cosine similarity closer to 1 indicates higher similarity (vectors pointing in the same direction), meaning the document is highly relevant to the query.

The documents are then ranked in descending order of their cosine similarity scores, presenting the most relevant results to the user first.

### **Advantages and Limitations**

**Advantages of TF-IDF:**

* **Improved Relevance:** By giving more weight to rare and important terms, TF-IDF often produces more relevant search results compared to simple Bag-of-Words counts.  
* **Handles Common Words:** The IDF component effectively down-weights common, less informative words, preventing them from dominating the similarity scores.  
* **Simplicity and Scalability:** It's relatively simple to compute and scales well to large document collections.

**Limitations of TF-IDF:**

* **Semantic Blindness:** Like the basic BoW model, TF-IDF still operates at the word level and completely ignores the semantic relationships between words (e.g., "car" and "automobile" are treated as distinct terms). It cannot understand context or nuances of language.  
* **Word Order Ignorance:** It does not consider the order of words, meaning phrases and the grammatical structure of sentences are lost.  
* **Sparse Vectors:** For very large vocabularies, TF-IDF vectors can be extremely high-dimensional and sparse (mostly zeros), which can be computationally inefficient for some operations.

Despite its limitations, TF-IDF remains a cornerstone in information retrieval, often used as a baseline or as a feature engineering technique in more complex systems. Its effectiveness and simplicity make it a valuable tool for understanding and processing text data.


In [7]:
# Building a TF-IDF Vectorizer based search engine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus['text'])


def search_tfidf(corpus, search_query):
    corpus = corpus.copy()
    search_query_vector = vectorizer.transform([search_query])
    similarity = cosine_similarity(search_query_vector, tfidf_matrix).flatten()
    corpus['similarity'] = similarity
    return corpus.sort_values(by='similarity', ascending=False)


print(search_tfidf(corpus, search_query).head(10).to_markdown())

|      | text                                                               |   similarity |
|-----:|:-------------------------------------------------------------------|-------------:|
|  699 | Finding the Best Photo                                             |     0.358477 |
| 1550 | What Can You Learn From Accelerometer Data                         |     0.319214 |
|  675 | Can a machine learn to teach?                                      |     0.30413  |
| 1764 | What can we learn from a movie's color?                            |     0.298678 |
| 1983 | Finding Your Way, Courtesy of Machine Learning                     |     0.285296 |
| 1520 | A novel way to Soccer Match Prediction                             |     0.260768 |
| 1991 | Learn To Rate Fine Food                                            |     0.233312 |
| 1882 | The Price is Right? Estimating Medical Costs with Machine Learning |     0.2305   |
|   26 | This Sentence is Not a Question, or Is It?                   

While TF-IDF (Term Frequency-Inverse Document Frequency) has been a cornerstone of information retrieval for decades, the Okapi BM25 (Best Match 25\) ranking function emerged as a more sophisticated and often more effective approach to calculating document relevance for a given search query. BM25 is a probabilistic retrieval model that builds upon the TF-IDF concept, but with significant improvements, particularly in how it handles term frequency saturation and document length normalization. It's widely used in modern search engines due to its empirical effectiveness.

### **How BM25 Works**

BM25 calculates a relevance score for each document with respect to a search query. This score is a sum of the scores for each term in the query. For a query containing terms q1​,…,qn​, the BM25 score for a document D is typically calculated as:

$$
Score(D, Q) = \sum_{i=1}^{n} IDF(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{avgdl}\right)}
$$
Let's break down the components:

1. **IDF (Inverse Document Frequency):** Similar to TF-IDF, BM25 uses IDF to weigh terms. Rare terms across the corpus contribute more to the total score than common terms. The formula for IDF in BM25 often uses a slightly different variant to the standard log function in TF-IDF, sometimes including smoothing terms to avoid division by zero for terms appearing in no documents. IDF(qi​) is the Inverse Document Frequency of term qi​.  
2. **Term Frequency (**f(qi​,D)**):** This represents how many times term qi​ appears in document D. Unlike raw TF, BM25 incorporates parameters k1​ and b to prevent term frequency from saturating.  
   * **Saturation:** In simple TF, a word appearing 100 times in a document contributes 100 times as much as a word appearing once. BM25 acknowledges that beyond a certain point, additional occurrences of a term provide diminishing returns to relevance. The k1​ parameter controls this saturation point. Higher values of k1​ lead to less saturation.  
   * The term f(qi​,D)+k1​f(qi​,D)⋅(k1​+1)​ is the core of how BM25 handles term frequency, allowing it to give more weight to more frequent terms up to a point, then the contribution levels off.  
3. **Document Length Normalization (**∣D∣ **and** avgdl**):** This part of the formula addresses the issue of longer documents potentially having higher raw term frequencies simply because they are longer.  
   * ∣D∣ is the length of the document D (e.g., in words).  
   * avgdl is the average document length in the corpus.  
   * The parameter b (often between 0 and 1\) controls the degree of document length normalization.  
     * If b=0, no length normalization is applied.  
     * If b=1, full length normalization is applied. The term (1−b+b⋅avgdl∣D∣​) scales the denominator, effectively reducing the impact of term frequency in longer documents relative to shorter ones, especially if the term is not particularly frequent in the longer document compared to its overall length.

### **BM25 for Search**

When a query is submitted, each document's BM25 score is calculated based on the query terms. Documents are then ranked by their BM25 scores, from highest to lowest, to present the most relevant results. The parameters k1​ (typically between 1.2 and 2.0) and b (typically around 0.75) are tunable and often optimized based on the specific characteristics of the document corpus and search task.

### **Advantages and Limitations**

**Advantages of BM25:**

* **Improved Relevance:** BM25 generally outperforms plain TF-IDF in most retrieval scenarios due to its refined handling of term frequency saturation and document length normalization.  
* **Probabilistic Foundation:** Its roots in probabilistic models provide a more theoretically sound basis for relevance estimation compared to the heuristic nature of raw TF-IDF.  
* **Robustness:** It is relatively robust and performs well across diverse datasets without extensive tuning.

**Limitations of BM25:**

* **Semantic Gap:** Like TF-IDF, BM25 is a keyword-matching algorithm. It still suffers from the "semantic gap" problem, meaning it doesn't understand synonyms (e.g., "car" and "automobile" are treated as distinct) or the deeper meaning and context of words.  
* **Word Order Ignorance:** It treats the query and documents as a "bag of words," ignoring the order of words and thus failing to capture phrases or idiomatic expressions.  
* **Parameter Tuning:** While robust, optimal performance can sometimes require careful tuning of k1​ and b for specific corpora.  
* **Outperformed by Modern Methods:** For complex search tasks requiring deep semantic understanding, BM25 is often surpassed by neural network-based models and dense retrieval methods (like those using word embeddings or transformer models) that capture contextual relationships.

Despite the emergence of more advanced neural retrieval methods, BM25 remains a highly effective, computationally efficient, and widely used ranking function in many practical search systems, often serving as a strong baseline or even a crucial component in hybrid retrieval architectures.


In [8]:
# rank_bm25 based search engine
from rank_bm25 import BM25Okapi
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')


def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stopwords
    tokens = [
        word for word in tokens if word not in stopwords.words('english')]
    # Remove special characters
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in tokens]
    # Remove empty strings
    tokens = [word for word in tokens if word != '']
    return tokens


corpus['tokens'] = corpus['text'].apply(preprocess_text)
corpus['tokens'].head(10).to_markdown()
# BM25
bm25 = BM25Okapi(corpus['tokens'].tolist())


def search_bm25(corpus, search_query):
    corpus = corpus.copy()
    search_query_tokens = preprocess_text(search_query)
    scores = bm25.get_scores(search_query_tokens)
    corpus['score'] = scores
    return corpus.sort_values(by='score', ascending=False)


print(search_bm25(corpus, search_query).head(10).to_markdown())

[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/google/home/vermavineet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/google/home/vermavineet/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


|      | text                                           | tokens                                                |    score |
|-----:|:-----------------------------------------------|:------------------------------------------------------|---------:|
| 1983 | Finding Your Way, Courtesy of Machine Learning | ['finding', 'way', 'courtesy', 'machine', 'learning'] | 10.7682  |
|  675 | Can a machine learn to teach?                  | ['machine', 'learn', 'teach']                         | 10.0588  |
|  699 | Finding the Best Photo                         | ['finding', 'best', 'photo']                          |  9.37019 |
| 1708 | Win Some, Learn Some                           | ['win', 'learn']                                      |  8.01062 |
| 1359 | Can Machines Learn Genres                      | ['machines', 'learn', 'genres']                       |  7.18643 |
|  885 | How well do people learn?                      | ['well', 'people', 'learn']                           |  7.18643 |


## Vector Embeddings on text

In the realm of natural language processing (NLP), computers don't inherently understand human language. Words, phrases, and documents are abstract concepts to a machine. **Vector embeddings** are a revolutionary technique that bridges this gap by converting textual data into dense numerical representations, or vectors, in a continuous vector space. These vectors capture the semantic meaning and contextual relationships of words and phrases, allowing machines to process and "understand" language in a way that was previously impossible with traditional methods like Bag-of-Words or TF-IDF.

### **What are Vector Embeddings?**

At its core, a vector embedding is an array of real numbers (e.g., `[0.1, -0.5, 0.8, 0.2, ...]`) that represents a piece of text. Each number in the array corresponds to a dimension in the vector space. The magic lies in how these numbers are assigned: words or phrases that are semantically similar tend to be located closer together in this high-dimensional space, while dissimilar terms are further apart. For instance, the vector for "king" might be close to "queen" and "man," and the vector difference between "king" and "man" could be similar to the difference between "queen" and "woman."

### **Why are They Needed?**

Traditional text representation methods like Bag-of-Words or TF-IDF suffer from significant limitations:

* **Lack of Semantic Understanding:** They treat words as independent entities, ignoring synonyms, polysemy, and the underlying meaning. "Car" and "automobile" would be seen as entirely different terms.  
* **High Dimensionality and Sparsity:** For large vocabularies, vectors can become extremely long and mostly filled with zeros, leading to inefficient computation and storage.  
* **No Contextual Information:** They cannot capture the meaning of a word based on its surrounding words.

Vector embeddings overcome these limitations by:

* **Capturing Semantic Similarity:** They embed words with similar meanings close to each other in the vector space.  
* **Capturing Relationships:** They can learn analogies and relationships between words.  
* **Reduced Dimensionality:** They represent text in dense, lower-dimensional vectors compared to sparse, high-dimensional representations.

### **How are Vector Embeddings Generated?**

Vector embeddings are typically learned from vast amounts of text data using various machine learning techniques:

1. **Context-Independent Embeddings (e.g., Word2Vec, GloVe):** These models learn a single, fixed embedding for each word, regardless of its context. They analyze the co-occurrence patterns of words in a large corpus. If words frequently appear together or in similar contexts, their embeddings will be closer.  
2. **Contextual Embeddings (e.g., BERT, GPT, Gemini Embeddings):** More advanced models, particularly those based on the Transformer architecture, generate embeddings that are *context-dependent*. This means the embedding for a word like "bank" would be different if it's used in the context of a "river bank" versus a "financial bank." These models process entire sentences or paragraphs to generate rich, context-aware representations for each word or even for the entire input.

### **How are Vector Embeddings Used in Search?**

Vector embeddings are crucial for **semantic search**, which goes beyond simple keyword matching:

1. **Vectorizing Documents:** All documents in a corpus are transformed into their corresponding embedding vectors using a pre-trained embedding model. These vectors are then stored in a specialized database called a **vector database** (or vector index).  
2. **Vectorizing Queries:** When a user submits a query, it is also converted into an embedding vector using the *same* embedding model.  
3. **Similarity Search:** Instead of matching keywords, the search engine now performs a **similarity search** in the vector space. It finds document vectors that are "closest" to the query vector. The most common metric for measuring this similarity is **Cosine Similarity**: Let and B be two embedding vectors.

$$
\text{Cosine Similarity}(A, B) = \frac{A \cdot B}{||A|| \cdot ||B||} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}
$$

A cosine similarity score close to 1 indicates high semantic similarity.

Documents are then ranked based on their cosine similarity scores to the query vector. This allows the search engine to retrieve documents that might not contain the exact keywords but are semantically related to the query, significantly improving search relevance and recall.

### **Conclusion**

Vector embeddings have transformed NLP, enabling machines to grasp the nuances of human language. They are a cornerstone of modern search, recommendation systems, clustering, and many other AI applications, pushing the boundaries of what's possible in understanding and interacting with text data.


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

from typing import List
from langchain_huggingface import HuggingFaceEmbeddings

HF_EMBEDDING_MODEL = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)


def generate_embeddings(contents: List[str], embedding_model=HF_EMBEDDING_MODEL):
    """
    Generate embeddings for the given content using the Google Gemini model.
    """
    response = embedding_model.embed_documents(contents)
    return np.array(response)


statements = [
    'The King won the battle but lost the war.',
    'The Man won the battle but lost the war.',
    'The Queen won the battle but lost the war.',
    'The Woman won the battle but lost the war.',
]

all_embeddings = generate_embeddings(statements)

similarity = cosine_similarity(all_embeddings, all_embeddings)

_statements = [statement.replace(
    'the battle but lost the war.', '...') for statement in statements]

df = pd.DataFrame(similarity, columns=_statements)

df.index = _statements

df.round(3)

,The King won ...,The Man won ...,The Queen won ...,The Woman won ...
The King won ...,1.000,0.830,0.866,0.712
The Man won ...,0.830,1.000,0.748,0.751
The Queen won ...,0.866,0.748,1.000,0.803
The Woman won ...,0.712,0.751,0.803,1.000


## Searching for similar vectors using Cosine Similarity

In the realm of modern information retrieval and recommendation systems, **vector embeddings** allow us to represent items (like text documents, images, or user preferences) as numerical vectors in a high-dimensional space. Once text is converted into these meaningful vector representations, the task of finding "similar" items transforms into a geometric problem: identifying vectors that are "close" to each other in this space. Among various distance or similarity metrics, **Cosine Similarity** stands out as a popular and effective choice for this purpose.

### **The Role of Cosine Similarity**

Cosine Similarity measures the cosine of the angle between two non-zero vectors. It determines whether two vectors are pointing in roughly the same direction, which indicates semantic similarity, regardless of their magnitude (length). A cosine similarity score ranges from \-1 to 1:

* **1:** Indicates identical direction (maximum similarity).  
* **0:** Indicates orthogonality (no correlation).  
* **\-1:** Indicates diametrically opposite direction (maximum dissimilarity).

The formula for cosine similarity between two vectors A and B is:

$$
\text{Cosine Similarity}(A, B) = \frac{A \cdot B}{||A|| \cdot ||B||}
$$

Here, A⋅B is the dot product of the vectors, and ∣∣A∣∣ and ∣∣B∣∣ are their respective magnitudes (Euclidean norms).

### **Searching for Similar Vectors**

The process of searching for similar vectors using cosine similarity typically involves:

1. **Embedding Generation:** A query (e.g., a search phrase) is first converted into its corresponding vector embedding using the same model used for embedding the documents or items in your database.  
2. **Similarity Calculation:** This query vector is then compared against the pre-computed embedding vectors of all documents or items in your collection. For each comparison, a cosine similarity score is calculated.  
3. **Ranking:** The documents/items are then ranked in descending order based on their cosine similarity scores, with those having the highest positive scores considered most similar and thus most relevant to the query.

This approach allows for semantic search, where results are based on meaning rather than just keyword matching, making it powerful for tasks like question answering, content recommendation, and plagiarism detection.


In [54]:
sq = 'The Soldier lost all wars'
search_embedding = generate_embeddings([sq])

similarity = cosine_similarity(
    search_embedding.reshape(1, -1), all_embeddings
)

scores = [(score, statement)
          for score, statement in zip(similarity[0].round(3), statements)]

best_matches = sorted(scores, reverse=True, key=lambda x: x[0])

best_matches

[(0.722, 'The Man won the battle but lost the war.'),
 (0.618, 'The King won the battle but lost the war.'),
 (0.583, 'The Woman won the battle but lost the war.'),
 (0.568, 'The Queen won the battle but lost the war.')]

In [11]:
# Lets load a larger corpus of text

# Vector Based Search Engine
from tqdm.auto import tqdm

tqdm.pandas(desc="Generating embeddings")


def search(
        query: str,
        vector_database: pd.DataFrame,
        number_of_results: int = 10,
        embedding_model=HF_EMBEDDING_MODEL
):
    """
    Search for the most similar content in the vector database.
    """
    search_embedding = generate_embeddings(
        [query],
        embedding_model=embedding_model
    )[0]
    results = vector_database.apply(
        lambda x: cosine_similarity(
            x.embedding.reshape(1, -1), search_embedding.reshape(1, -1)
        )[0][0], axis=1
    ).sort_values(ascending=False).head(number_of_results)

    return pd.DataFrame(
        {
            'text': vector_database.iloc[results.index].text,
            'similarity': results.values,
        }
    )


# Build In Memory Vector Database
vector_database = pd.DataFrame({'text': corpus.text, 'embedding': [
    i for i in generate_embeddings(corpus.text)]})

# Vector Based Search

results = search(query=search_query, vector_database=vector_database)

results

,text,similarity
1983,"Finding Your Way, Courtesy of Machine Learning",0.583221
1449,Machine Learning In JavaScript,0.561169
1780,How can machine learning help stock investment?,0.536195
606,Parallelizing Machine Learning Algorithms,0.524660
1981,Exploring the Efficacy of Machine Learning in ...,0.519362
675,Can a machine learn to teach?,0.519019
1359,Can Machines Learn Genres,0.517664
1225,Machine Learning in Stock Price Trend Forecasting,0.505323
413,An Empirical Study of Machine Learning Techniq...,0.504424
2240,Implementing Machine Learning to Earthquake En...,0.502205
